<a href="https://colab.research.google.com/github/AjitKabadi/Deep-Learning/blob/main/Cnn_FashionMnist_optimized%5BKerastuner%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 5.3MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=66845dc7cd8944271bab0154afa21079a710616092baaf6914a4b7b7bd23f801
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=1a258f8f67bb1bea5d61814669df8feb8964a4b60b27f0c1190b63c93db8c870
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np

In [4]:
print(tf.__version__)

2.4.1


In [5]:
fashion_mnist= keras.datasets.fashion_mnist

In [6]:
(train_images, train_labels), (test_images,test_labels)= fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [7]:
train_images=train_images/255.0
test_images=test_images/255.0

In [8]:
 train_images[0].shape

(28, 28)

In [9]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images= test_images.reshape(len(test_images),28,28,1)

In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [12]:
tuner_serach = RandomSearch(build_model,objective='val_accuracy',max_trials=5, directory='output',project_name='mnist_fashoin')

In [13]:
tuner_serach.search(train_images,train_labels, epochs=3, validation_split= 0.1)

Trial 5 Complete [00h 00m 23s]
val_accuracy: 0.871833324432373

Best val_accuracy So Far: 0.9143333435058594
Total elapsed time: 00h 01m 55s
INFO:tensorflow:Oracle triggered exit


In [15]:
model=tuner_serach.get_best_models(num_models=1)[0]

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        73792     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                2949200   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 3,025,082
Trainable params: 3,025,082
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(train_images,train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1104 - accuracy: 0.9579 - val_loss: 0.2766 - val_accuracy: 0.9117
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0758 - accuracy: 0.9710 - val_loss: 0.3123 - val_accuracy: 0.9143
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0458 - accuracy: 0.9850 - val_loss: 0.3572 - val_accuracy: 0.9153
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0333 - accuracy: 0.9885 - val_loss: 0.4153 - val_accuracy: 0.9165
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0254 - accuracy: 0.9907 - val_loss: 0.4789 - val_accuracy: 0.9130
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0183 - accuracy: 0.9938 - val_loss: 0.5400 - val_accuracy: 0.9120
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0172 - accuracy: 0.9941 - val_loss: 0.5561 - val_accuracy